# VFB_connect Quickstart Guide


In [1]:
!pip install -r ../../../requirements.txt --quiet
!pip install ../../../ --quiet

# Import lib and initialise object
from vfb_connect import vfb

Welcome to the Virtual Fly Brain API
See the documentation at: https://virtualflybrain.org/docs/tutorials/apis/

Establishing connections to https://VirtualFlyBrain.org services...
Session Established!

Type vfb. and press tab to see available queries. You can run help against any query e.g. help(vfb.get_TermInfo)



## VFB connect API overview

The VFB connect API provides programmatic access to the databases underlying [Virtual Fly Brain](http://virtualflybrain.org). 

At the core of Virtual Fly Brain is a set of curated terms for Drosophila neuro-anatomy organised into a queryable classification, including terms for brain regions, e.g. [nodulus](http://virtualflybrain.org/reports/FBbt_00003680) and neurons e.g. [MBON01](http://virtualflybrain.org/reports/FBbt_00100234). These terms are used to annotate and classify individual brain regions and neurons in images and connectomics data. For example the term MBON01 is used to classify individual [neurons from sources including the CATMAID-FAFB and Neuprint-HemiBrain databases](https://v2.virtualflybrain.org/org.geppetto.frontend/geppetto?q=FBbt_00100234,ListAllAvailableImages). VFB stores both [registered 3D images](http://virtualflybrain.org/reports/VFB_00101382) and [connectomics data](https://v2-dev.virtualflybrain.org/org.geppetto.frontend/geppetto?q=VFB_00101382,ref_neuron_neuron_connectivity_query) (where available) for all of these neurons.

A single VfbConnect object wraps connections and canned queries against all open VFB databases. It includes methods for retreiving metadata about anatomy, individual brain regions and neurons including IDs for these that can be used for queries against other databases (e.g. CATMAID & neuprint).  It provides methods for downloading images and connectomics data. It provides access to sophisticated queries for anatomical classes and individual neurons accoring to their classification & properties. 

### Locations for methods under a `VfbConnect` object.

1. Directly under `vfb` are:
   1. A set of methods that take lists of IDs as a primary argument and return metadata. 
   2. A set of methods for mapping between VFB IDs and external IDs
   3. A set of methods that take the names of classes in VFB e.g. 'nodulus' or 'Kenyon cell', or simple query expressions using the names of classes and return metadata about the classes.
   4. A set methods for querying connectivity and similarity
2. Direct access to API queries is provided under the 'nc' and 'oc' attributes for Neo4J and OWL queries respectively. We will not cover details of how to use these here.

Note: available methods and their documentation are easy to explore in DeepNote. Tab completion and type adhead can be used to help find methods. Float your cursor over a method to see its signature and docstring. 

### 1. `vfb` Neo4j query methods overview

**1.1** `vfb.get_TermInfo` TermInfo queries return the results of a VFB Term Information window as JSON, following the [VFB_JSON standard](https://virtualflybrain.github.io/VFB_json_schema/doc/schema_doc.html), or a summary that can be provided as a DataFrame or dictonary.

In [2]:
# A query for full TermInfo.  This probably produces more information than you will need for most purposes.
vfb.get_TermInfo(['FBbt_00003686'], summary=False)[0].keys()

dict_keys(['term', 'query', 'version', 'parents', 'relationships', 'related_individuals', 'xrefs', 'anatomy_channel_image', 'pub_syn', 'def_pubs', 'targeting_splits'])

In [3]:
# A query for summary info
summary = vfb.get_TermInfo(['FBbt_00003686'], return_dataframe=False)
summary


[{'label': 'Kenyon cell',
  'symbol': 'KC',
  'id': 'FBbt_00003686',
  'tags': ['Entity',
   'Anatomy',
   'Cell',
   'Class',
   'Nervous_system',
   'Neuron',
   'has_subClass',
   'hasScRNAseq'],
  'description': 'Intrinsic neuron of the mushroom body. They have tightly-packed cell bodies, situated in the rind above the calyx of the mushroom body (Ito et al., 1997). Four short fascicles, one per lineage, extend from the cell bodies of the Kenyon cells into the calyx (Ito et al., 1997). These 4 smaller fascicles converge in the calyx where they arborize and form pre- and post-synaptic terminals (Christiansen et al., 2011), with different Kenyon cells receiving input in different calyx regions/accessory calyces (Tanaka et al., 2008). They emerge from the calyx as a thick axon bundle referred to as the peduncle that bifurcates to innervate the dorsal and medial lobes of the mushroom body (Tanaka et al., 2008). Pre-synaptic terminals were identified using two presynaptic markers (Brp an

In [4]:
# The same method can be used to get info about individual neurons

summary = vfb.get_TermInfo(['VFB_jrchjrch'])
summary


,label,symbol,id,tags,description,parents_label,parents_id,data_source,accession,xrefs,templates,dataset,license
0,5-HTPLP01_R (FlyEM-HB:1324365879),5-HTPLP01_R,VFB_jrchjrch,"[Entity, Adult, Anatomy, Cell, Glutamatergic, ...","tracing status-Roughly traced, cropped-False","[adult glutamatergic neuron, adult serotonergi...","[FBbt_00058208, FBbt_00110945]",[neuprint_JRC_Hemibrain_1point2point1],[1324365879],"[neuronbridge:1324365879, neuprint_JRC_Hemibra...","[JRC2018Unisex, JRC_FlyEM_Hemibrain]",[Xu2020NeuronsV1point1],[https://creativecommons.org/licenses/by/4.0/l...


**1.2** The `vfb` also includes methods for mapping between IDs from different sources. 

In [5]:
# Some bodyIDs of HemiBrain neurons from the neuprint DataBase:
bodyIDs = [1068958652, 571424748, 1141631198]
vfb.xref_2_vfb_id(map(str, bodyIDs), return_just_ids=False) # Note IDs must be strings

{'1068958652': [{'db': 'neuronbridge', 'vfb_id': 'VFB_jrch06r9'},
  {'db': 'neuronbridge', 'vfb_id': 'VFB_jrchjwda'},
  {'db': 'neuprint_JRC_Hemibrain_1point0point1', 'vfb_id': 'VFB_jrch06r9'},
  {'db': 'neuprint_JRC_Hemibrain_1point2point1', 'vfb_id': 'VFB_jrchjwda'}],
 '571424748': [{'db': 'neuronbridge', 'vfb_id': 'VFB_jrch06r6'},
  {'db': 'neuronbridge', 'vfb_id': 'VFB_jrchjwct'},
  {'db': 'neuprint_JRC_Hemibrain_1point0point1', 'vfb_id': 'VFB_jrch06r6'},
  {'db': 'neuprint_JRC_Hemibrain_1point2point1', 'vfb_id': 'VFB_jrchjwct'}],
 '1141631198': [{'db': 'neuronbridge', 'vfb_id': 'VFB_jrch05uz'},
  {'db': 'neuronbridge', 'vfb_id': 'VFB_jrchjw8r'},
  {'db': 'neuprint_JRC_Hemibrain_1point0point1', 'vfb_id': 'VFB_jrch05uz'},
  {'db': 'neuprint_JRC_Hemibrain_1point2point1', 'vfb_id': 'VFB_jrchjw8r'}]}

In [ ]:
# xref queries can be constrained by DB. Results can optionally be reversed

vfb.xref_2_vfb_id(map(str, bodyIDs), db = 'neuprint_JRC_Hemibrain_1point2point1' , reverse_return=True, return_just_ids=False)

{'VFB_jrchjwct': [{'acc': '571424748',
   'db': 'neuprint_JRC_Hemibrain_1point2point1'}],
 'VFB_jrchjw8r': [{'acc': '1141631198',
   'db': 'neuprint_JRC_Hemibrain_1point2point1'}],
 'VFB_jrchjwda': [{'acc': '1068958652',
   'db': 'neuprint_JRC_Hemibrain_1point2point1'}]}

### 2. `vfb` OWL and Neo4j combined methods overview

**2.1** Methods that take the names of classes in VFB e.g. 'nodulus' or 'Kenyon cell', or simple query expressions using the names of classes and return metadata about the classes or individual neurons

In [7]:
KC_types = vfb.get_subclasses("Kenyon cell")
KC_types[0:5]

,label,symbol,id,tags,data_source,accession,parents_label,parents_id
0,Kenyon cell of main calyx,,FBbt_00047926,"[Entity, Adult, Anatomy, Cell, Cholinergic, Cl...",[],[],"[adult Kenyon cell, adult Kenyon cell, adult K...","[FBbt_00049825, FBbt_00049825, FBbt_00049825, ..."
1,alpha/beta core Kenyon cell,KCab-c,FBbt_00110929,"[Entity, Adult, Anatomy, Cell, Cholinergic, Cl...",[],[],"[alpha/beta surface/core Kenyon cell, alpha/be...","[FBbt_00049838, FBbt_00049838, FBbt_00049838, ..."
2,gamma main Kenyon cell,KCg-m,FBbt_00111061,"[Entity, Adult, Anatomy, Cell, Cholinergic, Cl...",[],[],"[Kenyon cell of main calyx, Kenyon cell of mai...","[FBbt_00047926, FBbt_00047926, FBbt_00047926, ..."
3,alpha/beta inner-core Kenyon cell,,FBbt_00049111,"[Entity, Adult, Anatomy, Cell, Cholinergic, Cl...",[],[],"[alpha/beta core Kenyon cell, alpha/beta core ...","[FBbt_00110929, FBbt_00110929]"
4,gamma Kenyon cell,,FBbt_00100247,"[Entity, Anatomy, Cell, Class, Nervous_system,...",[],[],"[Kenyon cell, Kenyon cell, Kenyon cell, Kenyon...","[FBbt_00003686, FBbt_00003686, FBbt_00003686, ..."


**2.2** Methods for querying connectivity

Please see [Connectomics Notebook](connectomics.html) for examples